<a href="https://colab.research.google.com/github/jabarragann/PytorchTutorialsJuanAntonio/blob/master/EEG_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting to Google drive

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# !ls
# !mkdir -p 'gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Data/D4/'
# !mkdir -p 'gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Data/D4/preprocess'
!ls
import os
os.chdir('/content')
!ls 

!pip install pycuda

Data
gdrive	RunTimeDir  sample_data
     |████████████████████████████████| 1.6MB 35.3MB/s 
     |████████████████████████████████| 61kB 26.2MB/s 
     |████████████████████████████████| 471kB 50.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/de/c2/d5/351a6b47b20d417e82a669cf53f8cb4d7b55a57f73cbd05184
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Stored in directory: /root/.cache/pip/wheels/cc/0e/59/0e7f24103d1ebce045037aa17b75548a8387f5e7d2d0011fdc
Successfully built pycuda pytools mako


#Setting up the Environment


1. Delete current run time directory
2. Create all needed directories
3. Copy data from google drive account to Colab directory





In [3]:
import os
import shutil

def createDirectories(rootPath, *args):
  
  print("Creating directories")
  for idx, path in enumerate(args):
    try:  
      os.makedirs(rootPath + path)
    except OSError:  
      print ("{:d}: Creation of the directory {} failed".format(idx,path))
    else:  
      print ("{:d}: Successfully created the directory {}".format(idx, path))
  print()

def copyFiles(srcRootPath,dstRootPath, *directories):
  print("copy data from google drive")
  for idx, path in enumerate(directories):
    print("{:d}: Copy files from: {:s}".format(idx,path) )
    completePath = srcRootPath+path
    srcFiles = os.listdir(completePath)
    srcFiles.sort()

    for file in srcFiles:
        srcPath = completePath + file
        dstPath = dstRootPath+path

        if os.path.isfile(srcPath):
            shutil.copy(srcPath, dstPath)
            
  print()

#Set /content as working directory
os.chdir('/content')

#Remove Previous RuntimeDirectory
try:
  shutil.rmtree('./RunTimeDir')
except:
  print("RunTimeDirectory not found")
else:
  print("Deleting previous RunTimeDirectory")
finally:
  print()


# Create new directories
rootPath = "./RunTimeDir/Data/"
directories = ['D1/preprocessed','D1/final',
               'D2/preprocessed','D2/final',
               'D3/preprocessed','D3/final',
               'D4/preprocessed','D4/final']

createDirectories(rootPath, *directories)


#Copy data from google drive to RuntimeDir

gdrivePath ='gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Data/'
srcDirectories = ['D1/preprocessed/']
dstRootPath = "./RunTimeDir/Data/"

copyFiles(gdrivePath, dstRootPath, *srcDirectories)


# Check current working directory
os.chdir('/content/RunTimeDir')
path = os.getcwd()  
print ("The current working directory is %s" % path, end='\n\n') 



RunTimeDirectory not found

Creating directories
0: Successfully created the directory D1/preprocessed
1: Successfully created the directory D1/final
2: Successfully created the directory D2/preprocessed
3: Successfully created the directory D2/final
4: Successfully created the directory D3/preprocessed
5: Successfully created the directory D3/final
6: Successfully created the directory D4/preprocessed
7: Successfully created the directory D4/final

copy data from google drive
0: Copy files from: D1/preprocessed/

The current working directory is /content/RunTimeDir



#Data Preprocessing

In [6]:
import pandas as pd
import numpy as np
import time
import pickle
from scipy import signal
import cv2
import os

class DataContainer:

    def __init__(self):
        self.dataWindowArray = []
        self.length = 0

        # Normalizing values of whole trial
        self.epocMean = -1
        self.epocStd = -1
        self.shimmerMean = -1
        self.shimmerStd = -1

    def createWindow(self,beginTime, endTime, shimmerSize, epocSize, shimmerScaling=None,
                                                                     epocScaling=None):
        self.dataWindowArray.append(DataWindow(beginTime, endTime, shimmerSize, epocSize,shimmerScaling, epocScaling))
        self.length += 1

    def fillData(self, shimmerData=None, epocData=None):
        #Fill Shimmer Data
        windowIdx = 0
        shimmerIdx = 0
        shimmerData = shimmerData[['PPG', 'COMPUTER_TIME', 'LABEL']].values
        for i in range(shimmerData.shape[0]):
            while shimmerData[i,-2] > self.dataWindowArray[windowIdx].endTime:
                windowIdx += 1
                shimmerIdx = 0

                if windowIdx >= self.length:
                    break

            if windowIdx >= self.length:
                break

            self.dataWindowArray[windowIdx].shimmerArray[shimmerIdx] = shimmerData[i,:]
            self.dataWindowArray[windowIdx].actualShimmerSize += 1
            shimmerIdx += 1

        #Fill Epoc Data
        windowIdx = 0
        epocIdx = 0
        epocData = epocData[['AF3', 'F7', 'F3', 'FC5', 'T7',
                             'P7', 'O1', 'O2', 'P8', 'T8',
                             'FC6', 'F4', 'F8', 'AF4', 'COMPUTER_TIME',
                             'LABEL']].values
       
                    
        for i in range(epocData.shape[0]):
          
            '''
            Get the correct initial window (This needs to be done because the secondary 
            task labels started before that the sensor streaming of data)
            '''
            while epocData[i,-2] > self.dataWindowArray[windowIdx].endTime:
                windowIdx += 1
                epocIdx = 0
            
                if windowIdx >= self.length:
                    break
            
            
            if windowIdx >= self.length:
                break
            
            '''
            Discard data that was taken before the first window of time
            '''
            if epocData[i,-2] > self.dataWindowArray[windowIdx].beginTime:
                self.dataWindowArray[windowIdx].epocArray[epocIdx] = epocData[i,:]
                self.dataWindowArray[windowIdx].actualEpocSize += 1
                epocIdx += 1

    def createMetrics(self):
            
        for i in range(self.length):
            self.dataWindowArray[i].calculateLabel()
            self.dataWindowArray[i].createSpectogramVolume()

    def normalizeData(self):
        for i in range(self.length):
            self.dataWindowArray[i].normalizeData()

    def dumpWindowsToFiles(self, destinationPath, trial):
        for i in range(self.length):
            self.dataWindowArray[i].createPickleFile(i, destinationPath, trial)



class DataWindow:

    def __init__(self, beginTime, endTime, shimmerSize, epocSize,shimmerScaling=None, epocScaling=None):

        self.beginTime = float(beginTime)
        self.endTime = float(endTime)

        self.shimmerArray = np.zeros((shimmerSize, 1+2))
        self.epocArray = np.zeros((epocSize, 14+2))
        self.actualShimmerSize = 0
        self.actualEpocSize = 0

        self.spectogramVolume = np.zeros((14,51,7))
        self.globalLabel = None

        self.shimmerMean = shimmerScaling[0]
        self.shimmerStd = shimmerScaling[1]
        self.epocMean = epocScaling[0]
        self.epocStd = epocScaling[1]


    def createSpectogramVolume(self):
        if self.actualEpocSize > 400:
            #Add EEG Spectogram
            for j in range(14):
                x = self.epocArray[:self.actualEpocSize, j]
                f, t, Sxx = signal.spectrogram(x, 1, nperseg=100, mode='magnitude')
                # print(Sxx.max())
                Sxx = cv2.resize(Sxx, dsize=(7, 51))
                self.spectogramVolume[j, :, :] = Sxx

            # #Add PPG Spectogram
            # if self.actualShimmerSize < 600:
            #     print(self.actualShimmerSize)
            #
            # x = self.shimmerArray[:self.actualShimmerSize, 0]
            # f, t, Sxx = signal.spectrogram(x, 1, nperseg=100, mode='magnitude')
            # Sxx = cv2.resize(Sxx, dsize=(7, 51))
            # self.spectogramVolume[14, :, :] = Sxx


    def normalizeData(self):
        self.epocArray[:,:14] = (self.epocArray[:,:14] - self.epocMean)/self.epocStd
        self.shimmerArray[:,0] = (self.shimmerArray[:,0] - self.shimmerMean)/self.shimmerStd

    def calculateLabel(self):
        totalLength = self.actualEpocSize + 1e-6
        label = sum(self.epocArray[:self.actualEpocSize, -1]) / totalLength
        self.globalLabel = round(label)

    def createPickleFile(self, windowIdx, destinationPath, trial):
        if self.actualEpocSize > 400 and windowIdx>4:
            finalDict = {'data': self.spectogramVolume, 'label': self.globalLabel}
            with open(destinationPath + 'S1_T{:d}_{:03d}.pickle'.format(trial, windowIdx), 'wb') as handle:
                pickle.dump(finalDict, handle, protocol=pickle.HIGHEST_PROTOCOL)


TRIAL = 7
DATASET = 1
import os

if __name__ == '__main__':
  
    os.chdir('/content/RunTimeDir')
    path = os.getcwd()  
    print ("The current working directory is %s" % path, end='\n\n') 

    destinationPath = "./Data/D{:d}/final/".format(DATASET)
    dataPath = "./Data/D{:d}/preprocessed/".format(DATASET)
    
    try:
        os.mkdir(destinationPath)
    except OSError:
        print("Creation of the directory %s failed" % destinationPath)
    else:
        print("Successfully created the directory %s " % destinationPath)

    
    for TRIAL in [3,4,5,6,7]:
        shimmerFile = pd.read_csv(dataPath + 'S1_T{:d}_fusion_shimmer.txt'.format(TRIAL), sep=' ')
        epocFile = pd.read_csv(dataPath + 'S1_T{:d}_fusion_epoc.txt'.format(TRIAL), sep=' ')
        fiveSecondWindowStamps = pd.read_csv(dataPath + "S1_T{:d}_TimestampEvery5Seconds.txt".format(TRIAL)
                                             , sep=' ').values

        #Get Normalizing Values
        shimmerData = shimmerFile['PPG'].values
        epocData = epocFile[['AF3', 'F7', 'F3', 'FC5', 'T7',
                             'P7', 'O1', 'O2', 'P8', 'T8',
                             'FC6', 'F4', 'F8', 'AF4']].values

        epocMean, epocStd = epocData.mean(), epocData.std()
        shimmerMean, shimmerStd = shimmerData.mean(), shimmerData.std()

        #Create and fill Data Container
        container = DataContainer()

        for i in range(fiveSecondWindowStamps.shape[0] - 1):
            container.createWindow(fiveSecondWindowStamps[i],
                                   fiveSecondWindowStamps[i+1],
                                   1000, 1000,
                                   shimmerScaling=(shimmerMean, shimmerStd),
                                   epocScaling=(epocMean,epocStd))

        container.fillData(shimmerData=shimmerFile, epocData = epocFile)
        container.normalizeData()
        container.createMetrics()
        container.dumpWindowsToFiles(destinationPath, TRIAL)

        print("Finish Creating Pickle Files from trial {:d}".format(TRIAL))


The current working directory is /content/RunTimeDir

Creation of the directory ./Data/D1/final/ failed
Finish Creating Pickle Files from trial 3
Finish Creating Pickle Files from trial 4
Finish Creating Pickle Files from trial 5
Finish Creating Pickle Files from trial 6
Finish Creating Pickle Files from trial 7


#Test GPU

In [7]:
import torch
import pycuda.driver as cuda


print("Do we have GPUs available?")
print(torch.cuda.is_available())

## Get Id of default device
cuda.init()
print("Default device: {:d}".format(torch.cuda.current_device()) )
print("GPU name: {:s}".format(cuda.Device(0).name()))

print("GPU name with Pytorch: {:s}".format(torch.cuda.get_device_name(0) ))


Do we have GPUs available?
True
Default device: 0
GPU name: Tesla T4
GPU name with Pytorch: Tesla T4


#Classification


Model

In [0]:
import torch.nn as nn

class ConvNetwork(nn.Module):

    def __init__(self, out_1=2, out_2=1, out_3=12, activation="relu"):

        super(ConvNetwork,self).__init__()

        #SET activation
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'selu':
            self.activation = nn.SELU()
        elif activation == 'elu':
            self.activation = nn.ELU()

        #First Convolutional Layer
        self.cnn1 = nn.Conv2d(in_channels=14,out_channels=out_1,kernel_size=1,padding=0)
        #self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.bn1 = nn.BatchNorm2d(out_1)
        self.relu1 = self.activation

        #Second Convolutional Layer
        self.cnn2 = nn.Conv2d(in_channels=out_1, out_channels=out_2, kernel_size=3, padding=2)
        self.bn2 = nn.BatchNorm2d(out_2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu2 = self.activation

        # Third Convolutional Layer
        self.cnn3 = nn.Conv2d(in_channels=out_2, out_channels=out_3, kernel_size=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_3)
        self.relu3 = self.activation

        #Fully connected Layer
        self.fc1 = nn.Linear(out_3*26*4, 784)
        self.fc_bn = nn.BatchNorm1d(784)
        self.fc2 = nn.Linear(784, 2)
        self.relu4 = self.activation

    def forward(self,x):

        out = self.cnn1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        #out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.bn2(out)
        out = self.maxpool2(out)
        out = self.relu2(out)

        out = self.cnn3(out)
        out = self.bn3(out)
        out = self.relu3(out)

        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.fc_bn(out)
        out = self.relu4(out)

        out = self.fc2(out)

        return out



EEG dataset Class


In [0]:
from torch.utils.data import Dataset, DataLoader
import torch
from os.path import isfile, join
from os import listdir
import pickle


class EegDataset(Dataset):

    def __init__(self,datasetPath="./Dataset/",transform =None):

        self.datasetPath = datasetPath
        self.files = [join(datasetPath, f) for f in listdir(datasetPath) if isfile(join(datasetPath, f))]
        self.len = len(self.files)

        self.x = torch.zeros((self.len, 14, 51, 7))
        self.y = torch.zeros(len(self.files),dtype=torch.long)

        self.positiveLength = 0
        self.negativeLength = 0
        self.positiveIdx = []
        self.negativeIdx = []

        for i in range(len(self.files)):

            with open(self.files[i],'rb') as f1:
                dataDict = pickle.load(f1)
                self.x[i, :, :, :] = torch.torch.from_numpy(dataDict['data'])
                self.y[i] = int(dataDict['label'])

                if self.y[i] == 1:
                    self.positiveLength += 1
                    self.positiveIdx.append(i)
                elif self.y[i] == 0:
                    self.negativeLength += 1
                    self.negativeIdx.append(i)


        # Normalizin Values
        self.xMean = self.x.mean()
        self.xStd  = self.x.std()
        self.arbitraryScaling = self.x.max()*3/8
        self.xMax = self.x.max()
        self.xMin = self.x.min()

        # Scaling by (max - min)
        self.x = self.x / (self.xMax - self.xMin + 1e-7)

        #Scaling by mean and std
        #self.x = (self.x - self.xMean) / (self.xStd + 1e-7)

        #Scaling with arbitrary scaling factor
        # self.x = self.x / (self.arbitraryScaling)


        self.transform = transform


    def __getitem__(self, index):

        sample = self.x[index], self.y[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.len


Classifier

In [20]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from os import listdir
from os.path import isfile, join
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import time


def save_checkpoint(optimizer, model, epoch, trLossH, valLossH, valAccH, filename):
    checkpoint_dict = {'optimizer': optimizer.state_dict(),
                       'model': model.state_dict(),
                       'epoch': epoch,
                       'trLossH': trLossH,
                       'valLossH': valLossH,
                       'valAccH': valAccH}

    torch.save(checkpoint_dict, filename)


def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename, map_location=DEVICE)
    epoch = checkpoint_dict['epoch']
    trLossH = checkpoint_dict['trLossH']
    valLossH = checkpoint_dict['valLossH']
    valAccH = checkpoint_dict['valAccH']

    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])

    return epoch, trLossH, valLossH, valAccH


def fiveKFoldTraining(trainData, folds, cnn, totalEpochs, modelNumb):

    valAccuracyPlots = []
    loss = 0
    for i in range(5):
        print("Start Training Fold {:d}".format(i))
        #Create Data loaders
        trainLoader = DataLoader(trainData, batch_size=16, sampler=folds[i]['train'])
        validLoader = DataLoader(trainData, batch_size=16, sampler=folds[i]['val'])

        #Create the model, optimizer and loss function
        torch.manual_seed(RANDOM_SEED)
        model = ConvNetwork( out_1=cnn['CONV_LAYER1'],
                             out_2=cnn['CONV_LAYER2'],
                             out_3=cnn['CONV_LAYER3'],
                             activation=cnn['ACTIVATION'])
        
        model.to(DEVICE)
        lossFunction = nn.CrossEntropyLoss()
        optimizer = optim.SGD(params=model.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)

        #TrainLogger
        valAccuracyHistory = []
        valLossHistory = []
        trLossHistory = []

        #Train Fold
        #Reset Max accuracy
        maxValidationAccuracy = 0.7

        for epoch in range(totalEpochs):
          
            #Train Model with training set
            model.train()
            for x, y in trainLoader:
                x, y = x.to(DEVICE), y.to(DEVICE)  
                optimizer.zero_grad()
                yHat = model(x)
                loss = lossFunction(yHat, y)
                loss.backward()
                optimizer.step()
                trLossHistory.append(loss.item())

            
            #Calculate Accuracy with validation set
            correct = 0
            total = 0
            
            model.eval()
            with torch.no_grad():
                for x, y in validLoader:
                    x, y = x.to(DEVICE), y.to(DEVICE)  
                    z = model(x)
                    yHat = z.argmax(1)
                    correct += (yHat == y).sum().item()
                    total += y.shape[0]

            accuracy = correct / total
            valAccuracyHistory.append(accuracy)
            
            #Print Accuracy
            if epoch%10 == 0 and epoch != 0:
                #print("Loss in Training in epoch {:d} set:       {:.5f}".format(epoch, loss))
                print("Max validation accuracy in epoch {:d} set: {:.5f}".format(epoch, max(valAccuracyHistory)))

            # Save Check Point
            if accuracy > maxValidationAccuracy:
                maxValidationAccuracy = accuracy
                checkpoint_filename = MODELS_DIRECTORY + 'Model{:d}/classifier-f{:02d}-e{:03d}.pkl'.format(modelNumb, i, epoch)
                save_checkpoint(optimizer, model, epoch, trLossHistory, valLossHistory, valAccuracyHistory,
                                checkpoint_filename)

                checkpoint_txt = MODELS_DIRECTORY + 'Model{:d}/classifier-f{:02d}-e{:03d}-acc{:0.5f}.txt'.format(modelNumb, i, epoch, maxValidationAccuracy)
                with open(checkpoint_txt,'w') as f:
                    pass

                maxValidationAccuracy = accuracy

        maxValAccPerFold =  max(valAccuracyHistory)
        valAccuracyPlots.append(valAccuracyHistory)
        print("Max Validation Accuracy fold {:02d}:        {:0.5f}\n".format(i, maxValAccPerFold))


    #Save data Accuracies
    statsFile = MODELS_DIRECTORY + 'Model{:d}/foldAccuracies.pkl'.format(modelNumb)
    with open(statsFile, 'wb') as f:
        pickle.dump(valAccuracyPlots, f)

    #Print training summary
    for i in range(5):
        print("Max accuracy in fold {:d}: {:.4f}".format(i,max(valAccuracyPlots[i])))

    averageMaxAcc = sum([max(valAccuracyPlots[i]) for i in range(5)])/5
    print("Average Max Accuracy: {:.5f}".format(averageMaxAcc))


def createModelDirectory(modelNumber):
    path = MODELS_DIRECTORY + "Model{:02d}".format(modelNumber)
    try:
        os.makedirs(path)
    except OSError:
        print("Creation of the directory %s failed" % path)
    else:
        print("Successfully created the directory %s " % path)


#Cnn Architectures -- Best Model cnn8!!
cnn1 = {"CONV_LAYER1": 32, "CONV_LAYER2": 64, "CONV_LAYER3": 12, "ACTIVATION": 'selu'}
cnn2 = {"CONV_LAYER1": 32, "CONV_LAYER2": 64, "CONV_LAYER3": 12, "ACTIVATION": 'elu'}
cnn3 = {"CONV_LAYER1": 32, "CONV_LAYER2": 64, "CONV_LAYER3": 12, "ACTIVATION": 'relu'}
cnn4 = {"CONV_LAYER1": 16, "CONV_LAYER2": 32, "CONV_LAYER3": 6, "ACTIVATION": 'selu'}
cnn5 = {"CONV_LAYER1": 16, "CONV_LAYER2": 32, "CONV_LAYER3": 6, "ACTIVATION": 'elu'}
cnn6 = {"CONV_LAYER1": 16, "CONV_LAYER2": 32, "CONV_LAYER3": 6, "ACTIVATION": 'relu'}
cnn7 = {"CONV_LAYER1": 64, "CONV_LAYER2": 128, "CONV_LAYER3": 24, "ACTIVATION": 'selu'}
cnn8 = {"CONV_LAYER1": 64, "CONV_LAYER2": 128, "CONV_LAYER3": 24, "ACTIVATION": 'elu'}
cnn9 = {"CONV_LAYER1": 64, "CONV_LAYER2": 128, "CONV_LAYER3": 24, "ACTIVATION": 'relu'}


#Global variables
DEVICE = 'cuda'
MODELS_DIRECTORY = './Models/'

#RandomSeed
#Model 2 and Model 3
#RANDOM_SEED = 742
#Model 4
RANDOM_SEED = 480


if __name__ == '__main__':

    #Create Training data object
    trainData = EegDataset(datasetPath="./Data/D1/final")
    print("Number of negative examples ",trainData.negativeLength)
    print("Number of positive examples ", trainData.positiveLength)

    #Shuffle training data
    split = int(0.2 * len(trainData))
    np.random.seed(seed=RANDOM_SEED)
    idxArr = np.random.permutation(len(trainData))

    #Create 5 folds for cross Validation
    folds = []
    total = len(trainData)

    for i in range(5):
        valRange = range(split*i, split*(i+1) if split*(i+1) < total else total)
        valIdx = idxArr[valRange]
        trainIdx = np.delete(idxArr, valRange)
        trainSampler = SubsetRandomSampler(trainIdx)
        valSampler = SubsetRandomSampler(valIdx)
        folds.append({'train': trainSampler, 'val': valSampler})

    #Train Models
    totalEpochs = 50
    print("Start training")
    #cnnArch = [cnn1,cnn2,cnn3,cnn4,cnn5,cnn6,cnn7,cnn8,cnn9]
    cnnArch = [cnn8]
    modelNumber = 31

    for cnn in cnnArch:
        print("Start Training Model {:d}".format(modelNumber))
        createModelDirectory(modelNumber)
        fiveKFoldTraining(trainData, folds, cnn, totalEpochs, modelNumber)
        modelNumber += 1



  

Number of negative examples  155
Number of positive examples  164
Start training
Start Training Model 31
Creation of the directory ./Models/Model31 failed
Start Training Fold 0
Max validation accuracy in epoch 10 set: 0.69841
Max validation accuracy in epoch 20 set: 0.69841
Max validation accuracy in epoch 30 set: 0.71429
Max validation accuracy in epoch 40 set: 0.76190
Max Validation Accuracy fold 00:        0.76190

Start Training Fold 1
Max validation accuracy in epoch 10 set: 0.76190
Max validation accuracy in epoch 20 set: 0.80952
Max validation accuracy in epoch 30 set: 0.80952
Max validation accuracy in epoch 40 set: 0.80952
Max Validation Accuracy fold 01:        0.80952

Start Training Fold 2
Max validation accuracy in epoch 10 set: 0.69841
Max validation accuracy in epoch 20 set: 0.79365
Max validation accuracy in epoch 30 set: 0.80952
Max validation accuracy in epoch 40 set: 0.80952
Max Validation Accuracy fold 02:        0.80952

Start Training Fold 3
Max validation accurac